In [30]:
# Installing all the necessary libraries we need
%pip install azure-keyvault-secrets
%pip install azure-identity
%pip install torch
%pip install torchvision

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [31]:
import os
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torchvision.models import densenet201


from __future__ import print_function, division
import numpy as np
import time
import copy
import mlflow

from azure.identity import AzureCliCredential, DefaultAzureCredential
from azure.keyvault.secrets import SecretClient
from azure.storage.filedatalake import DataLakeServiceClient

from azure.ai.ml import command, Input, MLClient
from azureml.core import Workspace, Dataset, Experiment, Run, ScriptRunConfig

In [32]:
cred = DefaultAzureCredential()
key_vault_name = 'serengetikeyvaultatqdhj2'
key_vault_url = f'https://{key_vault_name}.vault.azure.net/'
pool_conn_secret_name = 'DedicatedPool-odbc-ConnectionString'
adls_conn_secret_name = 'ADLS-ConnectionString'

secret_client = SecretClient(vault_url=key_vault_url, credential=cred)
adls_connection_string = secret_client.get_secret(adls_conn_secret_name).value

In [33]:
service_client = DataLakeServiceClient.from_connection_string(adls_connection_string)

file_system_name = "snapshot-serengeti"
file_path = "images/train"
file_system_client = service_client.get_file_system_client(file_system=file_system_name)
paths = file_system_client.get_paths(path=file_path)

for path in paths:
    print(path.name + "\n")

images/train/buffalo

images/train/buffalo/S3_J11_R11_IMAG1407.JPG

images/train/buffalo/S3_J11_R11_IMAG1408.JPG

images/train/buffalo/S3_J11_R11_IMAG1409.JPG

images/train/buffalo/S3_J11_R11_IMAG1410.JPG

images/train/buffalo/S3_J11_R11_IMAG1411.JPG

images/train/buffalo/S3_J11_R11_IMAG1412.JPG

images/train/buffalo/S3_J11_R11_IMAG1413.JPG

images/train/buffalo/S3_J11_R11_IMAG1577.JPG

images/train/buffalo/S3_J11_R11_IMAG1578.JPG

images/train/buffalo/S3_J11_R11_IMAG1579.JPG

images/train/buffalo/S3_J11_R11_IMAG1580.JPG

images/train/buffalo/S3_J11_R11_IMAG1581.JPG

images/train/buffalo/S3_J11_R11_IMAG1588.JPG

images/train/buffalo/S3_J11_R11_IMAG1589.JPG

images/train/buffalo/S4_B07_R1_IMAG0371.JPG

images/train/buffalo/S4_B07_R1_IMAG0372.JPG

images/train/buffalo/S4_B07_R1_IMAG0373.JPG

images/train/buffalo/S4_B07_R1_IMAG0374.JPG

images/train/buffalo/S4_B07_R1_IMAG0375.JPG

images/train/buffalo/S4_B07_R1_IMAG0376.JPG

images/train/buffalo/S4_B07_R1_IMAG0441.JPG

images/train/buffal

In [34]:
from azureml.core import Workspace, Datastore, Dataset
# Get Azure Machine Learning workspace and experiment
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name='transfer-learning')

# Get Azure Machine Learning run context
run = Run.get_context()

datastore = Datastore.get(ws, 'snapshot_serengeti')

# Define data reference
data_reference = datastore.path('images')



In [35]:
from azureml.core import Dataset

data_folder = os.path.join(os.getcwd(), 'data')
os.makedirs(data_folder, exist_ok=True)

serengeti_dataset = Dataset.get_by_name(ws, name='Serengeti-Data')
serengeti_dataset.download(target_path='.', overwrite=False)

serengeti_dataset = serengeti_dataset.register(workspace=ws,
                                                 name='serengeti',
                                                 description='training and test dataset',
                                                 create_new_version=True)


UserErrorException: UserErrorException:
	Message: File "/mnt/batch/tasks/shared/LS_root/mounts/clusters/mlcomputebethany/code/Users/engbethany4/train/buffalo/S3_J11_R11_IMAG1407.JPG" already exists. Set overwrite=True to overwrite it,                             or choose an empty target path.
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "File \"/mnt/batch/tasks/shared/LS_root/mounts/clusters/mlcomputebethany/code/Users/engbethany4/train/buffalo/S3_J11_R11_IMAG1407.JPG\" already exists. Set overwrite=True to overwrite it,                             or choose an empty target path."
    }
}

In [40]:
import torchvision

train_set = torchvision.datasets.ImageFolder(root="./train", transform=transforms.ToTensor())
test_set = torchvision.datasets.ImageFolder(root="./val", transform=transforms.ToTensor())

# Load the training and test data
train_loader = DataLoader(train_set, batch_size=4, shuffle=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=4, shuffle=False, num_workers=2)

In [ ]:
# Set up data
train_data = Dataset.File.from_files(path=(datastore, 'images/train/')).as_named_input('train_data')
val_data = Dataset.File.from_files(path=(datastore, 'images/val/')).as_named_input('val_data')

In [41]:
# Define environment
myenv = "azureml://registries/azureml/environments/AzureML-ACPT-pytorch-1.11-py38-cuda11.3-gpu/versions/11"

# Define compute target
compute_target = "mlcomputebethany"

In [81]:
# Define training script and parameters
job = command(
    inputs=dict(
        num_epochs=50, learning_rate=0.001, momentum=0.9, output_dir="./outputs"
    ),
    compute=compute_target,
    environment=myenv,
    code="./src",  # location of source code
    command="python train.py --num_epochs ${{inputs.num_epochs}} --output_dir ${{inputs.output_dir}}",
    experiment_name="transfer-learning",
    display_name="pytorch-serengeti",
)

In [82]:
ml_client = MLClient(
    credential=cred,
    subscription_id="b4ddefc7-2d33-4fb2-b656-b3239eb6b0d0",
    resource_group_name="Serengeti",
    workspace_name="SerengetiAMLatqdhj24jh5y6",
)
ml_client.jobs.create_or_update(job)

Uploading src (0.02 MBs): 100%|██████████| 20924/20924 [00:00<00:00, 413093.04it/s]




Experiment,Name,Type,Status,Details Page
transfer-learning,salmon_feather_4qzqmny72z,command,Starting,Link to Azure Machine Learning studio


In [83]:
# tuning hyperparameters
from azure.ai.ml.sweep import Uniform

# we will reuse the command_job created before. we call it as a function so that we can apply inputs
job_for_sweep = job(
    learning_rate=Uniform(min_value=0.0005, max_value=0.005),
    momentum=Uniform(min_value=0.9, max_value=0.99),
)



In [84]:
# Define early termination policy
from azure.ai.ml.sweep import BanditPolicy
early_termination_policy = BanditPolicy(slack_factor=0.15, evaluation_interval=1, delay_evaluation=5)

sweep_job = job_for_sweep.sweep(
    compute=compute_target,
    sampling_algorithm="random",
    primary_metric="best_val_acc",
    goal="Maximize",
    max_total_trials=2,
    max_concurrent_trials=2,
    early_termination_policy=BanditPolicy(
        slack_factor=0.15, evaluation_interval=1, delay_evaluation=10
    ),
)

In [85]:
returned_sweep_job = ml_client.create_or_update(sweep_job)

# stream the output and wait until the job is finished
ml_client.jobs.stream(returned_sweep_job.name)

# refresh the latest status of the job after streaming
returned_sweep_job = ml_client.jobs.get(name=returned_sweep_job.name)

RunId: neat_drop_flll61gmvg
Web View: https://ml.azure.com/runs/neat_drop_flll61gmvg?wsid=/subscriptions/b4ddefc7-2d33-4fb2-b656-b3239eb6b0d0/resourcegroups/Serengeti/workspaces/SerengetiAMLatqdhj24jh5y6

Streaming azureml-logs/hyperdrive.txt

[2023-03-23T22:43:07.433850][GENERATOR][INFO]Trying to sample '2' jobs from the hyperparameter space
[2023-03-23T22:43:07.9780706Z][SCHEDULER][INFO]Scheduling job, id='neat_drop_flll61gmvg_1' 
[2023-03-23T22:43:07.9843982Z][SCHEDULER][INFO]Scheduling job, id='neat_drop_flll61gmvg_0' 
[2023-03-23T22:43:07.916989][GENERATOR][INFO]Successfully sampled '2' jobs, they will soon be submitted to the execution target.
[2023-03-23T22:43:08.8359608Z][SCHEDULER][INFO]Successfully scheduled a job. Id='neat_drop_flll61gmvg_1' 
[2023-03-23T22:43:08.8691719Z][SCHEDULER][INFO]Successfully scheduled a job. Id='neat_drop_flll61gmvg_0' 
[2023-03-23T22:44:07.193072][GENERATOR][INFO]Max number of jobs '2' reached for experiment.
[2023-03-23T22:44:07.313214][GENERATOR

In [ ]:
from azure.ai.ml.entities import Model

if returned_sweep_job.status == "Completed":

    # First let us get the run which gave us the best result
    best_run = returned_sweep_job.properties["best_child_run_id"]

    # lets get the model from this run
    model = Model(
        # the script stores the model as "outputs"
        path="azureml://jobs/{}/outputs/artifacts/paths/outputs/".format(best_run),
        name="run-model-example",
        description="Model created from run.",
        type="custom_model",
    )

else:
    print(
        "Sweep job status: {}. Please wait until it completes".format(
            returned_sweep_job.status
        )
    )

In [ ]:
import uuid

# Creating a unique name for the endpoint
online_endpoint_name = "aci-serengeti-endpoint-" + str(uuid.uuid4())[:8]

In [ ]:
from azure.ai.ml.entities import ManagedOnlineEndpoint

# create an online endpoint
endpoint = ManagedOnlineEndpoint(
    name=online_endpoint_name,
    description="Snapshot Serengeti data using transfer learning with PyTorch",
    auth_mode="key",
    tags={"data": "serengeti", "method": "transfer learning", "framework": "pytorch"},
)

endpoint = ml_client.begin_create_or_update(endpoint).result()

print(f"Endpoint {endpoint.name} provisioning state: {endpoint.provisioning_state}")

In [ ]:
endpoint = ml_client.online_endpoints.get(name=online_endpoint_name)

print(
    f'Endpint "{endpoint.name}" with provisioning state "{endpoint.provisioning_state}" is retrieved'
)

In [ ]:
from azure.ai.ml.entities import (
    ManagedOnlineDeployment,
    Model,
    Environment,
    CodeConfiguration,
)

online_deployment_name = "aci-serengeti"

# create an online deployment.
blue_deployment = ManagedOnlineDeployment(
    name=online_deployment_name,
    endpoint_name=online_endpoint_name,
    model=model,
    environment=myenv,
    code_configuration=CodeConfiguration(code="./score/", scoring_script="score.py"),
    instance_type="Standard_NC6s_v3",
    instance_count=1,
)

blue_deployment = ml_client.begin_create_or_update(blue_deployment).result()